In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [4]:
df1=pd.read_csv("YoutubeCommentsDataSet.csv")

In [6]:
df1.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [12]:
print(df1['Sentiment'].value_counts())

print(df1.isnull().sum())

Sentiment
positive    11432
neutral      4638
negative     2338
Name: count, dtype: int64
Comment      44
Sentiment     0
dtype: int64


In [14]:
df = df1.dropna()

In [16]:
print(df['Sentiment'].value_counts())

print(df.isnull().sum())

Sentiment
positive    11402
neutral      4625
negative     2337
Name: count, dtype: int64
Comment      0
Sentiment    0
dtype: int64


In [18]:
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    
    return text

# Apply cleaning to the dataset
df['cleaned_comment'] = df['Comment'].apply(clean_text)

/var/folders/ry/02hdr_2d74dfl2c2wtlr22sm0000gn/T/ipykernel_43346/2301877028.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_comment'] = df['Comment'].apply(clean_text)


In [20]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['sentiment_label'] = label_encoder.fit_transform(df['Sentiment'])

/var/folders/ry/02hdr_2d74dfl2c2wtlr22sm0000gn/T/ipykernel_43346/546517371.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment_label'] = label_encoder.fit_transform(df['Sentiment'])


In [22]:
from sklearn.model_selection import train_test_split

X = df['cleaned_comment']
y = df['sentiment_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Train the model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train_tfidf, y_train)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(multi_class='multinomial')

In [28]:
y_pred = model.predict(X_test_tfidf)

In [30]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.7514293493057447
              precision    recall  f1-score   support

    negative       0.60      0.31      0.41       441
     neutral       0.61      0.57      0.59       912
    positive       0.81      0.91      0.86      2320

    accuracy                           0.75      3673
   macro avg       0.67      0.60      0.62      3673
weighted avg       0.74      0.75      0.74      3673



In [36]:
results_df = pd.DataFrame({
    'Comment': X_test, 
    'Actual_Sentiment': y_test,  
    'Predicted_Sentiment': y_pred 
})
# Map numerical labels back to their original sentiment names
results_df['Actual_Sentiment'] = results_df['Actual_Sentiment'].map({0: 'Positive', 1: 'Neutral', 2: 'Negative'})
results_df['Predicted_Sentiment'] = results_df['Predicted_Sentiment'].map({0: 'Positive', 1: 'Neutral', 2: 'Negative'})

print(results_df.head(10))

                                                 Comment Actual_Sentiment  \
11851                              yellow rainbow friend          Neutral   
4003   first 500 cube unboxing second 300 cube unboxi...         Negative   
7857   ok think gorloss favorite admin right like he ...         Negative   
4724   20713 omg treated david rare voice emphatic un...          Neutral   
3001                                baa baa ba baa baaaa          Neutral   
8725   reading doesnt matter right planet wish wonder...         Negative   
11243  wasnt left school discovered interesting learn...         Negative   
9262                oh incredible street food look yummy         Negative   
8355                                               전 한국팬          Neutral   
16801  inspirationalthanks maker movie bringing life ...         Negative   

      Predicted_Sentiment  
11851             Neutral  
4003              Neutral  
7857             Negative  
4724             Negative  
3001        